In [2]:
import requests
import itertools
from arcgis.geometry import Geometry

import pandas as pd
from arcgis.features import SpatialDataFrame as SDF
from arcgis.features import GeoAccessor

from arcgis.gis import GIS, Item

import sys
sys.path.append('..')

import geometry_monkeypatch
import reach_tools

In [3]:
reach_item_id = 'c72a6e88a67e4eafb7b291f17ec0cf18'
reach_layer_url = 'https://services5.arcgis.com/12oODIpfxlRR11MF/arcgis/rest/services/Reach_Line/FeatureServer/0'
reach_id_ldub = 2156
reach_id_farmies = 2269
reach_id_canyon = 3066

In [158]:
import importlib
importlib.reload(reach_tools)

<module 'reach_tools' from '../reach_tools.py'>

In [159]:
gis = GIS(url='https://jdm1cc.maps.arcgis.com', username='joel_mccune', password='K3mosabe')
reach_layer = reach_tools.ReachFeatureLayer(reach_layer_url, gis)

In [162]:
reach_layer.flush()

{'addResults': [],
 'updateResults': [],
 'deleteResults': [{'objectId': 61,
   'uniqueId': 61,
   'globalId': None,
   'success': True},
  {'objectId': 73, 'uniqueId': 73, 'globalId': None, 'success': True},
  {'objectId': 98, 'uniqueId': 98, 'globalId': None, 'success': True},
  {'objectId': 78, 'uniqueId': 78, 'globalId': None, 'success': True},
  {'objectId': 96, 'uniqueId': 96, 'globalId': None, 'success': True},
  {'objectId': 67, 'uniqueId': 67, 'globalId': None, 'success': True},
  {'objectId': 77, 'uniqueId': 77, 'globalId': None, 'success': True},
  {'objectId': 91, 'uniqueId': 91, 'globalId': None, 'success': True},
  {'objectId': 82, 'uniqueId': 82, 'globalId': None, 'success': True},
  {'objectId': 66, 'uniqueId': 66, 'globalId': None, 'success': True},
  {'objectId': 88, 'uniqueId': 88, 'globalId': None, 'success': True},
  {'objectId': 54, 'uniqueId': 54, 'globalId': None, 'success': True},
  {'objectId': 76, 'uniqueId': 76, 'globalId': None, 'success': True},
  {'object

In [167]:
for reach_id in range(0, 10000):
    print('reach_id -', reach_id)
    reach = reach_tools.Reach.get_from_aw(reach_id)
    if type(reach) == reach_tools.Reach:
        trace_status = reach.update_putin_takeout_and_trace()
        if trace_status == True:
            resp = reach.publish(gis, reach_layer)
            print(resp)
        else:
            print('/no trace')
    else:
        print('\tno reach')

reach_id - 0
	no reach
reach_id - 1


TypeError: 'NoneType' object is not subscriptable

In [183]:
reach = reach_tools.Reach.get_from_aw(reach_id_ldub)
reach

reach_id                                                             2156
reach_name                                      Willard to Columbia River
reach_name_alternate                                                     
river_name                                            Little White Salmon
river_name_alternate                                                     
error                                                                None
notes                                                                    
difficulty                                                              V
difficulty_minimum                                                   None
difficulty_maximum                                                      V
difficulty_outlier                                                   None
abstract                Washington's best known class V run flowing ye...
description             The Little White Salmon is a highlight of Paci...
update_aw                             

In [184]:
reach.reach_points

[reach_id                                                          2156
 point_type                                                      access
 subtype                                                          putin
 name                                                              None
 side_of_river                                                     None
 nhdplus_measure                                                   None
 nhdplus_reach_id                                                  None
 collection_method                                                 None
 update_date                                                       None
 notes                                                             None
 description                                                       None
 SHAPE                {'x': -121.634402, 'y': 45.794848, 'spatialRef...
 uid                                   b67186b807104f41ba963c4595177dc1
 dtype: object,
 reach_id                                       

In [176]:
web_map = GIS().map()
web_map

MapView(layout=Layout(height='400px', width='100%'))

In [177]:
web_map.draw(reach.putin.geometry)
web_map.draw(reach.takeout.geometry)

In [180]:
resp_json = waters._get_point_indexing(reach.putin.geometry.x, reach.putin.geometry.y)

In [181]:
resp_json['output'] == None

True

In [194]:
importlib.reload(reach_tools)
reach = reach_tools.Reach.get_from_aw(reach_id_ldub)

In [195]:
import numpy as np
np.mean([reach.putin.geometry.x, reach.takeout.geometry.y])

-37.9577925

In [196]:
reach.centroid

{'x': -121.639992, 'y': 45.7568325, 'spatialReference': {'wkid': 4326}}

In [197]:
web_map.draw(reach.centroid)